In [39]:
import pandas as pd
import re
import random
import math

# LOAD THE DATA

- last column of df , wich is the  the variable of interest for classification will be renamed as "label";

- remove the missing values in df

In [40]:
def remover_colunas_irrelevantes(df, limite_percentual, palavras_chave_id=['id']):
    df_limpo = df.copy()
    colunas_a_remover = []

    for coluna in df.columns:
        # Cálculo da proporção de valores únicos
        proporcao_valores_unicos = df[coluna].nunique() / df[coluna].count()
    

        # Verificação da proporção e do nome da coluna
        if proporcao_valores_unicos > limite_percentual:
            colunas_a_remover.append(coluna)
            
        
        # Verificação se o nome da coluna contém alguma das palavras chave para ID
        coluna_lower = coluna.lower()
        if any(re.fullmatch(rf'\b{palavra}\b', coluna_lower) for palavra in palavras_chave_id):
            if coluna not in colunas_a_remover:
                colunas_a_remover.append(coluna)
                

    # Remove as colunas identificadas
    df_limpo.drop(colunas_a_remover, axis=1, inplace=True)
    

    return df_limpo

In [41]:
def df_to_data_matrix(df):
    return df.values.tolist()


# TRAIN AND TEST SPLIT

In [42]:
def train_test_split(df, test_size):
    # If test_size is a float, convert it to an integer representing the number of test samples
    if isinstance(test_size, float):
        test_size = int(round(test_size * len(df)))

    # Shuffle the dataset
    random.shuffle(df)
    
    # Calculate the split index using the integer test_size
    split_index = len(df) - test_size
    
    # Split the data into train and test sets
    train_data = df[:split_index]
    test_data = df[split_index:]
    
    return train_data, test_data

In [43]:
def k_fold_cross_validation(data_matrix, k):
    # Shuffle data_matrix in place
    random.shuffle(data_matrix)
    
    fold_size = len(data_matrix) // k
    folds = []
    
    # Create the k folds
    for i in range(k):
        start = i * fold_size
        if i == k - 1:  # Ensure the last fold includes the remainder
            end = len(data_matrix)
        else:
            end = start + fold_size

        # Select the test set as the current fold
        test_set = data_matrix[start:end]
        
        # Create training set as all the data except the current test set
        train_set = data_matrix[:start] + data_matrix[end:]
        
        folds.append((train_set, test_set))
    
    return folds

In [44]:
'''
cv_folds = k_fold_cross_validation(data_matrix, k=5)

for idx, (train, test) in enumerate(cv_folds):
    print(f"Fold {idx+1}")
    print("Train Set:")
    print(train)
    print("Test Set:")
    print(test)
    print("-" * 40)
    
'''

'\ncv_folds = k_fold_cross_validation(data_matrix, k=5)\n\nfor idx, (train, test) in enumerate(cv_folds):\n    print(f"Fold {idx+1}")\n    print("Train Set:")\n    print(train)\n    print("Test Set:")\n    print(test)\n    print("-" * 40)\n    \n'

# CLASS NODE

In [45]:
class Node:
    def __init__(self, attribute = None, label = None, is_leaf = False, count = {}):
        self.attribute = attribute
        self.label = label
        self.is_leaf = is_leaf
        self.count = {}
        self.children = {}

    def recursive(self, node, space):
        result = ""
        if node.is_leaf:
                if isinstance(node.count, dict):
                    count_str = " ".join(f"{count}" for _, count in node.count.items())
                else:
                    count_str = str(node.count)
                result += f"{space}{node.label}({count_str})\n"
        else:
            sorted_children = sorted(node.children.items(), key = lambda x: x[0]) # ordenar os filhos por ordem crescente
            for value, child in sorted_children: # percorrer os filhos
                result += f"{space}{node.attribute}:{value}\n"
                result += self.recursive(child, space + "  ")
        return result

    def __str__(self):
        return self.recursive(self, " ")


# ENTROPY AND SPLITS CALCULATIONS

In [46]:
def _count_classes(dataset):
    class_counts = {}
    for data in dataset:
        label = data[-1]
        if label in class_counts:
            class_counts[label] += 1
        else:
            class_counts[label] = 1
    return class_counts

In [47]:
def most_common_class(labels):
    class_counts = {}
    for label in labels:
        if label not in class_counts:
            class_counts[label] = 1
        else:
            class_counts[label] += 1
    return max(class_counts,key = class_counts.get)

In [48]:
def entropy(data_label):
    count_classes = {}
    total = len(data_label)
    entropy = 0

    for value in data_label:
        if value not in count_classes:
            count_classes[value] = 1
        else:
            count_classes[value] += 1
    for value in count_classes.values():
        p = value / total
        entropy += -p * math.log2(p)
    return entropy


In [49]:
def attribute_entropy(data_matrix, attributes):
    entropy_attributes = {}
    for _, val in attributes.items():
        unique_attributes = list(set(row[val] for row in data_matrix))
        entropy_attribute = 0
        for value in unique_attributes:
            subset = []
            for row in data_matrix:
                if row[val] == value:
                    subset.append(row[-1])
            if subset:  # Ensure subset is not empty before calculating its entropy
                subset_entropy = entropy(subset)
                subset_probability = len(subset) / len(data_matrix)
                entropy_attribute += subset_entropy * subset_probability
        entropy_attributes[val] = entropy_attribute
    return entropy_attributes

In [50]:
def information_gain(data_matrix,attributes, entropy_class):
        info_gain = None
        best_value = 0
        value = 0
        for key, val in attributes.items():
            entropy_class = entropy([linha[-1] for linha in data_matrix])
            entropy_attributes = attribute_entropy(data_matrix, attributes)
            value = entropy_class - entropy_attributes[val] # calcular o ganho de informação
            if value > best_value: # se o ganho de informação for maior que o anterior
                best_value = value # atualizar o melhor ganho de informação
                info_gain = key # atualizar o melhor atributo
        return info_gain

In [51]:
def split_info_gain(dataset, label_menor, label_maior):
        parent_entropy = entropy(dataset)
        probabily_menor = len(label_menor) / len(dataset)
        probabily_maior = len(label_maior) / len(dataset)
        entropy_menor = entropy(label_menor)
        entropy_maior = entropy(label_maior)
        value_info_gain = parent_entropy - (probabily_menor * entropy_menor) - (probabily_maior * entropy_maior)
        return value_info_gain

In [52]:
def calculate_best_split(data_matrix, atributes, best_attribute):
    unique_values = list(set([linha[attributes[best_attribute]] for linha in data_matrix]))
    best_split = ''
    best_info_gain = -99999
    
    if len(unique_values) == 1:
        return unique_values[0]
    
    for value in unique_values:
        
        is_bigger_subset = []
        is_lower_subset = []
        
        for linha in data_matrix:
            if float(linha[attributes[best_attribute]]) <= float(value):
                is_bigger_subset.append(linha)
            if float(linha[attributes[best_attribute]]) > float(value):
                is_lower_subset.append(linha)
                    
        labels_menor = [linha[-1] for linha in is_bigger_subset]
        labels_maior = [linha[-1] for linha in is_lower_subset]
        
        info_gains  = split_info_gain([linha[-1] for linha in data_matrix], labels_menor, labels_maior)
        if info_gains > best_info_gain:
            best_info_gain = info_gains
            best_split = value
            
    return best_split

# TREE IMPLEMENTATION


In [53]:
def determine_data_type(values):
    """ Determine if the attribute values are numeric or categorical. """
    try:
        [float(value) for value in values]
        return 'numeric'
    except ValueError:
        return 'categorical'

In [54]:
class Tree:
    def __init__(self, data_matrix, attributes, label_class):
        self.dataset = data_matrix
        self.classe = label_class # nome da coluna da classe
        self.entropy_class = entropy([linha[-1] for linha in data_matrix]) # entropia da classe
        self.entropy_atrributes = {} # dicionario com a entropia de cada atributo
        self.attribute_to_index = {attr: i for i, attr in enumerate(attributes)}

        self.attributes =   {}
        self.reverse_attributes = {} # exemplo : {1: 'sepallength', 2: 'sepalwidth', 3: 'petallength', 4: 'petalwidth', 5: 'class'}
        for key, position in attributes.items():
            if  position != len(attributes) -1:
                self.reverse_attributes[position] = key
                self.attributes[key] = position

        self.attribute_entropy = attribute_entropy(data_matrix, attributes)
        self.root = self.build_tree(data_matrix, self.attributes) # construir a arvore de decisão

    def build_tree(self, data_matrix, attributes):
        labels = [linha[-1] for linha in data_matrix]  # Classes of examples

        if len(set(labels)) == 1:
            return Node(attribute=None, is_leaf=True, label=labels[0], count=_count_classes(data_matrix))

        if not attributes:
            return Node(attribute=None, is_leaf=True, label=most_common_class(labels), count=len(labels))

        best_attribute = information_gain(data_matrix, attributes, self.entropy_class)
        root = Node(best_attribute)
        val = attributes[best_attribute]
        unique_values = list(set(linha[val] for linha in self.dataset))

        data_type = determine_data_type(unique_values)

        if data_type == 'numeric':
            best_split = calculate_best_split(data_matrix, attributes, best_attribute)
            subset_lower = [linha for linha in data_matrix if float(linha[val]) <= float(best_split)]
            subset_higher = [linha for linha in data_matrix if float(linha[val]) > float(best_split)]
            new_attributes = attributes.copy()
            del new_attributes[best_attribute]
            root.children['<= ' + str(best_split)] = self.build_tree(subset_lower, new_attributes)
            root.children['> ' + str(best_split)] = self.build_tree(subset_higher, new_attributes)
        else:
            for value in unique_values:
                subset = [linha for linha in data_matrix if linha[val] == value]
                if subset:
                    new_attributes = attributes.copy()
                    del new_attributes[best_attribute]
                    child_node = self.build_tree(subset, new_attributes)
                    root.children[value] = child_node
                else:
                    root.children[value] = Node(attribute=None, is_leaf=True, label=most_common_class(labels), count=0)

        return root
    
    def tranform(self, data):
        if not isinstance(data[0], list):  # Check if data is not already a list of lists
            data = [data]  # Make it a list of lists if it's a single row

        results = []
        for row in data:
            result = self.tranform_tree(self.root, row)
            results.append(result)
        return results

    def tranform_tree(self, tree, row):
        if tree.is_leaf:
            return tree.label

        #print("TREE.ATRIBUTTE = ", tree.attribute)
        #print("ROW = " , row)
        #print("TREE.CHILDREN = ", tree.children)
        
        attribute_index = self.attribute_to_index[tree.attribute]  # Convert attribute name to index
        attribute_value = row[attribute_index]
        
    

        try:
            attribute_value = float(attribute_value)
            #print(str(attribute_value) + " " + "Success")
            split_key = list(tree.children.keys())[0]
            split_operator, split_value = split_key.split(' ')
            subtree = tree.children

            if split_operator == '<=':
                try:
                    if float(attribute_value) <= float(split_value):
                        subtree = subtree['<= ' + split_value]
                    else:
                        subtree = subtree['> ' + split_value]
                except ValueError:
                    return None  # Non-numeric value, skip the comparison
            elif split_operator == '>':
                try:
                    if float(attribute_value) > float(split_value):
                        subtree = subtree['> ' + split_value]
                    else:
                        subtree = subtree['<= ' + split_value]
                except ValueError:
                    return None  # Non-numeric value, skip the comparison

            return self.tranform_tree(subtree, row)
        except ValueError:
            if attribute_value in tree.children:
                child = tree.children[attribute_value]
                return self.tranform_tree(child, row)
            else:
                # Handle missing attribute value or unseen attribute value
                return None


    def __str__(self):
        return str(self.root)

In [55]:

def df_to_data_matrix(df):
    return df.values.tolist()

# Read the CSV file into a DataFrame
data = pd.read_csv('connect4.csv')
print(data.head())

# Assuming the class label is in the last column and the ID column is absent in your CSV


attributes = {col: idx for idx, col in enumerate(data.columns[:-1])}
data_matrix = df_to_data_matrix(data)

# Initialize and build the tree
#tree = Tree(data_matrix, attributes, df.columns[-1])
#print(tree)
#print(data_matrix)

  a1 a2 a3 a4 a5 a6 b1 b2 b3 b4  ... f4 f5 f6 g1 g2 g3 g4 g5 g6  class
0  b  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b    win
1  b  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b    win
2  b  b  b  b  b  b  o  b  b  b  ...  b  b  b  b  b  b  b  b  b    win
3  b  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b    win
4  o  b  b  b  b  b  b  b  b  b  ...  b  b  b  b  b  b  b  b  b    win

[5 rows x 43 columns]


In [56]:
import random

def k_fold_cross_validation(data_matrix, k, attributes):
    random.shuffle(data_matrix)
    fold_size = len(data_matrix) // k
    accuracies = []

    for i in range(k):
        start = i * fold_size
        end = len(data_matrix) if i == k - 1 else start + fold_size
        test_set = data_matrix[start:end]
        train_set = data_matrix[:start] + data_matrix[end:]

        # Train the tree using train_set
        tree = Tree(train_set, attributes, df.columns[-1])
    
        # Test the tree using test_set
        correct_predictions = 0
       
        for row in test_set:
            
            
            
            prediction = tree.tranform([row[:-1]])  # Exclude the label
            if prediction[0] == row[-1]:  # Compare the predicted label with the actual label
                correct_predictions += 1

        accuracy = correct_predictions / len(test_set)
        accuracies.append(accuracy)

    average_accuracy = sum(accuracies) / k
    return average_accuracy , accuracies


In [57]:
average_accuracy, accuracies = k_fold_cross_validation(data_matrix, 3 , attributes)
print(average_accuracy)
print(accuracies)

NameError: name 'df' is not defined

In [ ]:
import random

def leave_one_out_cross_validation(data_matrix, attributes):
    random.shuffle(data_matrix)  # Embaralhando para randomizar a ordem dos dados
    accuracies = []

    # Loop sobre o dataset inteiro, deixando um de fora cada vez para teste
    for i in range(len(data_matrix)):
        test_set = [data_matrix[i]]  # O conjunto de teste é apenas um elemento
        train_set = data_matrix[:i] + data_matrix[i+1:]  # Todos os outros dados formam o conjunto de treino

        # Treinar a árvore usando o conjunto de treino
        tree = Tree(train_set, attributes, df.columns[-1])  # Supondo que a classe Tree e df estão definidos corretamente
        
        # Testar a árvore usando o conjunto de teste
        correct_predictions = 0
        
        for row in test_set:
            print("row:", row)
            prediction = tree.tranform([row[:-1]])  # Exclui a label dos dados de entrada
            print("prediction:", prediction[0])
            print("actual:", row[-1])
            if prediction[0] == row[-1]:  # Compara a label prevista com a label atual
                correct_predictions += 1
        
        # Calculando a acurácia para este caso de teste
        accuracy = correct_predictions / len(test_set)
        accuracies.append(accuracy)

    # Calcular a porcentagem de previsões corretas
    correct_percentage = accuracies.count(1.0) / len(accuracies) * 100
    return correct_percentage

# Exemplo de chamada à função
# Supondo que data_matrix e attributes estão definidos



In [ ]:
accuracy = leave_one_out_cross_validation(data_matrix, attributes)
print("Accuracy:", accuracy)
